## Import Libraries

In [1]:
from neo4j import GraphDatabase, basic_auth

## Setting Up Connections

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../../Encryption/Blank-Sandbox.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")
database = config.get("NEO4J_DATABASE", "neo4j")

## Building Nodes

### Driver Initialisation

In [5]:
driver = GraphDatabase.driver(
    uri,
    auth=basic_auth(username, password),
)

### Create Nodes

In [6]:
with driver.session() as session:
    session.run(
        """
        MATCH (n) DETACH DELETE n
        CREATE (:Topic {name: "AI"})
        CREATE (:Topic {name: "Climate"})
        CREATE (:Topic {name: "Sustainability"})
        """    
    )

In [7]:
with driver.session() as session:
    result = session.run(
        """
        MATCH (t:Topic)
        RETURN t.name AS name
        """
    )
    topics = list(set([record["name"] for record in result]))

print(topics)  # Optional: to see the result

['AI', 'Sustainability', 'Climate']


In [8]:
with driver.session() as session:
    result = session.run(
        """
        WITH [
            {name: 'AI', description: 'Artificial Intelligence and Machine Learning'},
            {name: 'Climate', description: 'Climate Change and Environmental Impact'},
            {name: 'Sustainability', description: 'Sustainable Development Goals and Practices'}
        ] AS topicData
        UNWIND topicData AS t
        MATCH (topic:Topic {name: t.name})
        WITH topic, t
        SET topic.description = t.description
        RETURN DISTINCT topic.name AS Name, topic.description AS Description
        """
    )

    for record in result:
        name = record["Name"]
        description = record["Description"]
        print(f"Name: {name}, Description: {description}")

Name: AI, Description: Artificial Intelligence and Machine Learning
Name: Climate, Description: Climate Change and Environmental Impact
Name: Sustainability, Description: Sustainable Development Goals and Practices
